# Correlations between transcriptome and OTU microbiome data

Notebook to run correlations between the host transcriptome data and OTUs from Dr. Wallace [manuscript](https://apsjournals.apsnet.org/doi/full/10.1094/PBIOMES-02-18-0008-R).